In [1]:
#Libraries
#Helpers

#EDA initial
#Preprocessing
#Eda Final
#Second Preprocessing

#Preprocessing Pipeline
#Model Core Pipeline
#Model Output Pipeline

#Testing


## Helpers

In [3]:

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [4]:

def coordinates(zipcode):
    zipcode = search.by_zipcode(zipcode)
    community = zipcode.post_office_city
    return community, "{},{}".format(zipcode.lat, zipcode.lng)

def creating_coordinates(df, column):
    df["Community"] = ""
    df["Coordinates"] = "None, None"
    for zipcode in df[column].unique():
        community, coordinate = coordinates(zipcode)
        df.loc[df[column] == zipcode, "Coordinates"] =  str(coordinate)
        df.loc[df[column] == zipcode, "Community"] =  community
    return df

In [5]:
def user_exist(user):
    if len(users.loc[users['UserID'] == user]) == 0:
        return False
    return True

def has_coordinates(user):
    c = users.loc[users['UserID'] == user, "Coordinates"] 

    if (len(c) == 0) or (c.iloc[0].split(',')[0]) == 'None':
        return False
    return True
    


In [6]:
def user_profile(user_id):
    user_profile = np.array(users.loc[users['UserID'] == user_id, ['DegreeType', 'Major']])[0]
    historical_apps = user_history.loc[user_history.UserID == user_id, 'JobTitle']
    user_profile = """
    Degree Type: {}
    Major: {}
    """.format(user_profile[0], user_profile[1])

    return user_profile

def historical_application(user_id):
    historical_apps = apps.loc[apps.UserID == user_id]['JobID']
    content = list()
    for application in historical_apps:
        temp = jobs.loc[jobs.JobID == application, ['Title', 'Description', 'Requirements']]
        if len(temp) != 0:
            content += [temp.Title.values + ". " + temp.Description.values + ". " + temp.Requirements.values]
    return content


In [8]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.epoch += 1
        

### Libraries

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

import pickle

from uszipcode import SearchEngine
from  geopy.distance import distance

import gc
import re

import warnings; warnings.simplefilter('ignore')

#### Checking Cores

In [9]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "SLOW VERSION"

In [10]:
MAX_DISTANCE = 15

#!dir .\Data\*.tsv

### Reading Data

In [11]:
folder = 'Data'

apps = pd.read_csv('./'+folder+'/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./'+folder+'/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./'+folder+'/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./'+folder+'/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('./'+folder+'/test_users.tsv', delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


### EDA 1

In [12]:
apps.head()


,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [13]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [14]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [15]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [16]:
test_users.head()
len(users)

389708

### ETL Pipeline - Preprocessing

#### Subsetting & Sampling  NY

In [17]:
users = users.loc[users.State == 'NY']
len(users)

20590

In [18]:
users = users.sample(frac=0.2, replace=False, random_state=1)
len(users)

4118

In [19]:
jobs = jobs.loc[jobs.State == 'NY']
all_jobs = jobs.copy()
#jobs.to_csv("NYjobs.tsv",  sep='\t',encoding='utf-8')

#### Subsetting jobs with zipcode for testing distance feature

In [20]:
jobs.Zip5 = jobs.Zip5.fillna(0)
jobs.Zip5 = jobs.Zip5.astype(int)
jobs = jobs.loc[jobs['Zip5'] != 0]

In [21]:
len(jobs)

33679

In [22]:
jobs = jobs.sample(frac=0.2, replace=False, random_state=1)
len(jobs)

6736

#### Clearning Memory

In [23]:
gc.collect()

0

#### Creating jobs coordinates and Communities

In [24]:
search = SearchEngine(simple_zipcode=True)

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [25]:

search.by_zipcode("2e").lat

In [26]:
%%timeit
jobs["Community"] = ""
jobs["Coordinates"] = ""
for zipcode in jobs.Zip5.unique():
    community, coordinate = coordinates(zipcode)
    jobs.loc[jobs.Zip5 == zipcode, "Coordinates"] =  str(coordinate)
    jobs.loc[jobs.Zip5 == zipcode, "Community"] =  community

788 ms ± 44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Creating user coordinates

In [27]:
users = creating_coordinates(users, 'ZipCode')

#### Cleaning Description and Requirements

In [28]:
jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)

In [29]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)

#### Making jobs profiles

In [30]:
jobs['profile'] = jobs['Title'].astype(str)  +  '. ' + jobs['Requirements'].astype(str) +  '. ' + jobs['Description'].astype(str) 

#### Making users profiles

In [31]:
%%timeit

users['profile'] =  (users['Major'].astype(str) +  " " ) + (users['DegreeType'].astype(str) + " ")

#Cleaning Garbage Words
users['profile'] = users['profile'].str.replace('None.', ' ')
users['profile'] = users['profile'].str.replace('Not Applicable', ' ')
users['profile'] = users['profile'].str.replace('nan', ' ') 


8.67 ms ± 835 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### EDA 2

#### Jobs by Communmties/County

In [32]:
jobs.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
272,"New York, NY",2079
335,"Rochester, NY",329
47,"Buffalo, NY",241
45,"Brooklyn, NY",228
43,"Bronx, NY",165
243,"Melville, NY",141
0,"Albany, NY",109
417,"White Plains, NY",109
374,"Syracuse, NY",91
163,"Hauppauge, NY",75


#### Jobs by zipcodes

In [33]:
jobs.groupby(['Community', 'Zip5', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Zip5,Coordinates,Count
386,"New York, NY",10001,"40.75,-73.99",633
399,"New York, NY",10017,"40.75,-73.973",212
398,"New York, NY",10016,"40.74,-73.97",143
354,"Melville, NY",11747,"40.79,-73.4",141
416,"New York, NY",10036,"40.76,-73.99",134
400,"New York, NY",10018,"40.76,-73.99",118
404,"New York, NY",10022,"40.76,-73.97",97
390,"New York, NY",10005,"40.705,-74.005",90
83,"Brooklyn, NY",11201,"40.7,-73.99",87
401,"New York, NY",10019,"40.77,-73.99",76


#### Users by Community

In [34]:
users.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
49,"Brooklyn, NY",544
284,"New York, NY",446
46,"Bronx, NY",405
354,"Rochester, NY",200
397,"Staten Island, NY",90
208,"Jamaica, NY",88
458,"Yonkers, NY",62
51,"Buffalo, NY",61
138,"Flushing, NY",38
11,"Astoria, NY",36


### Machine Learninig Pipeline: NLP Model

In [35]:
%%time
def similarities_nlp_model( model_name = "jobs_doc2vec_model",
                 mapping_name = "jobID_mapping.p", max_epochs = 100,
                 alpha = 0.025):
    
    document = list()
    jobID_mapping = dict()
        
    for i, token in enumerate(jobs['profile']):
        value = jobs.iloc[i]["JobID"]
        tokens = TaggedDocument(simple_preprocess(token), [i])
        document.append(tokens)
        jobID_mapping[i] = value

    epoch_logger = EpochLogger()
    model = Doc2Vec(size = 20, alpha=alpha, 
                    min_alpha=0.00025, min_count=2,
                    callbacks=[epoch_logger], dm =1, workers=8, window=2)
    
    model.build_vocab(document)
    print(model.corpus_count)
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(document, 
                    total_examples=model.corpus_count, 
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    # create a dictionary
    pickle.dump(model, open(model_name, "wb")) 
    pickle.dump(jobID_mapping, open(mapping_name, "wb")) 

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
    return model, jobID_mapping

model, jobID_mapping = similarities_nlp_model()


TypeError: __init__() got an unexpected keyword argument 'size'

### Recommender Pipeline

#### Recomender by popularity/Cold Star

In [46]:
unique = 0.3 #update name
top = 20

def ranking_by_popularity(top = 100):
    
    popular_jobs = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    ranking =  dict()
    top_i = 0
    
    while True:
        job_title = popular_jobs['JobTitle'].iloc[top_i]
        jobs_list = jobs.loc[jobs['Title'] == job_title, ['JobID']]['JobID'].unique().tolist()

        if len(jobs_list) > 1:
            ranking[job_title] = jobs_list

        if len(ranking) == top:
            break

        top_i +=1
    return ranking

ranking_popular = ranking_by_popularity(top = 50)

In [37]:

def recommender_popular_jobs(user_id,  top = 10):
    
    recommended_popular_jobs = dict()
    c1 = users.loc[users['UserID'] == user_id, 'Coordinates']
    count = 0
    if user_exist(user_id) and has_coordinates(user_id):
        for title, jobs_list in ranking_popular.items():
            
            distances = dict()
        
            for job in jobs_list:
                
                c2 = jobs.loc[jobs['JobID'] == job, 'Coordinates']
                if c2.iloc[0].split(',')[0] == 'None':
                    continue

                distances[job] = round(distance(c1, c2.iloc[0]).miles, 2)
            
            distances = sorted(distances.items(), key=lambda kv: kv[1])
            closest = distances[0]
            if closest[1] >= MAX_DISTANCE:
                continue
            recommended_popular_jobs[title] = (closest[0], closest[1])
            count += 1
            if count == top:
                break
            
        if len (recommended_popular_jobs) < top:
            count = len (recommended_popular_jobs) -1
            
            for title, jobs_list in ranking_popular.items():
                recommended_popular_jobs[title] = jobs_list[0]
                count += 1
                if count == top:
                    break

            
    else:
        for title, jobs_list in ranking_popular.items():
            recommended_popular_jobs[title] = jobs_list[0]
            count += 1
            if count == top:
                break
            
    
    
    return recommended_popular_jobs




#### Content Based Recommender



In [45]:
#If user has coordinates
#if user has info then similarties if not popularity ranker
jobID_mapping = pickle.load(open("jobID_mapping.p", "rb"))
model  = pickle.load(open("jobs_doc2vec_model", "rb"))

def content_distance_based_recommender(user_id, jobID_mapping = jobID_mapping, model =  model, top = 10):
    #As infer_vector produce stochastics result I made a for to save the best list
    user_profile = np.array(users.loc[users['UserID'] == user_id, 'profile'])[0]

    historical_apps = historical_application(user_id)
    for application in historical_apps:
        user_profile += ". " + (str(application[0]) + " ")
    user_profile = simple_preprocess(user_profile)
    
    best = 0
    tops =pd.DataFrame(index = range(top), columns = ['JobID', 'Title', 'Distance', 'Description', 'Requirements'])
    c1 = users.loc[users['UserID'] == user_id, 'Coordinates']
    job_distance_list = list()
    
    for i in range (1):
        inferred_vector = model.infer_vector(user_profile)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        sum_results = 0
        count = 0
        total_recom = 0
        job_distance_list = list()
        
        while True:
            job_id = jobID_mapping[sims[count][0]]
            c2 = jobs.loc[jobs['JobID'] == jobID_mapping[sims[count][0]], 'Coordinates']
            count +=1
            if len(c2) == 0:
                #print("Empty", sims[count][0])
                continue
            
            if c2.iloc[0].split(',')[0] == 'None':
                #print('None', sims[count][0])
                continue
                
            job_distance = round(distance(c1, c2).miles, 2)
            
            if job_distance >= MAX_DISTANCE:
                #print('Distance', job_distance,  sims[count][0])
                continue

            sum_results+=sims[count][1]
            total_recom +=1
            job_distance_list.append(job_distance)
            if total_recom == top:
                break
        
        ##Best simulation
        if sum_results > best:
            best = sum_results
            best_sim = sims
            job_distance_list2 = job_distance_list
            
    for i in range(top):
        recomendation = jobID_mapping[best_sim[i][0]]
        tops.iloc[i]['JobID', 'Title',  'Description', 'Requirements'] = np.array(jobs.loc[jobs['JobID'] == recomendation][['JobID', 'Title', 
                                                               'Description', 'Requirements']])[0]
        tops.iloc[i]['Distance'] = job_distance_list2[i]
    
    return tops 
 
        

FileNotFoundError: [Errno 2] No such file or directory: 'jobID_mapping.p'

In [41]:
def hybrid_recommender(user_id, top=10):
    
    if has_coordinates(user_id) and len(historical_application(user_id)) > 0:
        print("Content")
        return content_distance_based_recommender(user_id, top = top)
    else:
        print("Cold Star")
        return recommender_popular_jobs(user_id,  top = top)
    
    


### Testing

In [39]:
users.head(5)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99",Electrical Engineering
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93",High School
57797,1122774,1,Train,New Hartford,NY,US,13413,None,Business Administration,1987-01-01 00:00:00,2,23.0,Yes,Yes,8,"New Hartford, NY","43.06,-75.27",Business Administration
60141,1165147,1,Train,Staten Island,NY,US,10314,Bachelor's,BA/Criminal Justice,1980-06-01 00:00:00,7,35.0,Yes,Yes,200,"Staten Island, NY","40.6,-74.15",BA/Criminal Justice Bachelor's
371308,875025,7,Train,Brooklyn,NY,US,11219,None,Business Management,2009-01-01 00:00:00,8,6.0,Yes,Yes,8,"Brooklyn, NY","40.63,-74.0",Business Management


In [51]:
#Cold 153286, 944873
#Content 248008, 1184529
user_id = 944873

print("USER PROFILE: ", user_profile(user_id))
print("HISTORICAL APPLICATION CONTENT:\n\n", historical_application(user_id))
hybrid_recommender(user_id, top = 10)

USER PROFILE:  
    Degree Type: PhD
    Major: Industrial and Systems Engineering
    
HISTORICAL APPLICATION CONTENT:

 []
Cold Star


{'Customer Service Representative': 1015707,
 'Cashier': 668223,
 'Administrative Assistant': 617073,
 'Sales Associate': 407797,
 'Assistant Manager': 317522,
 'Office Manager': 203695,
 'Manager': 1097462,
 'Receptionist': 518270,
 'Customer Service': 119569,
 'Store Manager': 1075099}

#### How many user with content?

In [43]:
counter = 0
for u in users.UserID.unique():
    if len(historical_application(u)) > 0:
        counter += 1


In [44]:
print(counter)
counter/len(users.UserID.unique())

1204


0.2923749392909179